In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" -q

In [2]:
!pip install --no-deps "trl<0.9.0" peft accelerate bitsandbytes xformers datasets -q
!pip install triton

In [3]:
!pip install torch>=2.0.0 transformers>=4.30.0 peft>=0.3.0 datasets>=2.10.0 trl>=0.4.1 bitsandbytes>=0.39.0 accelerate>=0.20.0 scipy scikit-learn

In [4]:
!pip show huggingface_hub

Name: huggingface-hub
Version: 0.26.1
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /home/ubuntu/dev/Resume-analysis-LLM/.venv/lib/python3.10/site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, datasets, peft, tokenizers, transformers, unsloth_zoo


In [6]:
!pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 10.9 MB/s eta 0:00:00a 0:00:01


In [1]:
# Import the notebook_login function
from huggingface_hub import notebook_login

# Login to Hugging Face
notebook_login()

### training 

In [2]:
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel, unsloth_save_model
from trl import SFTTrainer
from transformers import TrainingArguments
from huggingface_hub import notebook_login
import os

# Login to Hugging Face
notebook_login()

# Check if dataset files exist
train_file_path = "reformatted_train.jsonl"
valid_file_path = "reformatted_valid.jsonl"

if not os.path.exists(train_file_path):
    print(f"File not found: {train_file_path}. Please ensure the file is in the correct directory.")
if not os.path.exists(valid_file_path):
    print(f"File not found: {valid_file_path}. Please ensure the file is in the correct directory.")

# Load dataset
dataset = load_dataset("json", data_files={"train": train_file_path, "validation": valid_file_path})

# Print sample to inspect structure
print("Dataset structure:")
print(dataset["train"][0])  # Print first example to see the structure

# Model setup
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

# PEFT setup
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=42,
    max_seq_length=max_seq_length
)

# Training setup
training_arguments = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    optim="adamw_torch",
    save_steps=500,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    gradient_checkpointing=True,
    max_steps=100,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    push_to_hub=True,
    hub_model_id="gunzzz24/TinyLlama-1.1B-Chat-v1.0",
)

def formatting_func(example):
    """
    Format the example based on your actual data structure.
    Modify this function according to your JSONL file structure.
    """
    # Example structure - modify according to your actual data format
    try:
        # If your data has 'input' and 'output' fields
        if 'input' in example and 'output' in example:
            return f"Input: {example['input']}\nOutput: {example['output']}"
        
        # If your data has 'resume' and 'analysis' fields
        elif 'resume' in example and 'analysis' in example:
            return f"Resume: {example['resume']}\nAnalysis: {example['analysis']}"
            
        # If your data has a different structure, modify accordingly
        else:
            # Print the keys available in the example to help debug
            print("Available keys in example:", example.keys())
            # Return the first available field as a fallback
            first_key = list(example.keys())[0]
            return str(example[first_key])
            
    except Exception as e:
        print(f"Error formatting example: {e}")
        print(f"Example structure: {example}")
        return ""

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=256,
    formatting_func=formatting_func
)

# Train
trainer.train()

# Save model
unsloth_save_model(model, tokenizer, "TinyLlama-1.1B-Chat-v1.0")

Dataset structure:
{'context': 'Resume Information:\n\nEducation:\n- Artificial Intelligence: September 2019, September 2024\n- Big Data &: September 2019, September 2024\n- Data Analysis: September 2019, September 2024\n- Digital Transformation Consulting: September 2019, September 2024\n- Scikit -Learn: September 2019, September 2024\n- KPI: September 2019, September 2024\n- PowerBI: September 2019, September 2024\n- Czech Technical University: September 2019, September 2024\n- Telecommunication Systems and Networks: September 2019, September 2024, February 2021 – June 2021\n- Cyber Security: September 2019, September 2024, February 2021 – June 2021\n- Programming: September 2019, September 2024, February 2021 – June 2021\n- PROFESSIONAL: September 2019, September 2024, February 2021 – June 2021\n\nExperience:\n- Ledger: July 2022, September 2023\n- IBM: July 2022, September 2023\n- NFT Paris: July 2022, September 2023\n- VivaTech: July 2022, September 2023\n- CoinShares: July 2022, 

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [3]:
!pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 6.9 MB/s eta 0:00:00a 0:00:01


### training to store as gguf

In [ ]:
import torch
from datasets import load_dataset
from unsloth import FastLanguageModel, unsloth_save_model
from trl import SFTTrainer
from transformers import (
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer
)
from huggingface_hub import notebook_login, upload_file, create_repo
import os
import subprocess
from ctransformers import AutoModelForCausalLM as CTModel
import logging
import json
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ModelTrainer:
    def __init__(
        self,
        base_model_name="unsloth/gemma-2b-bnb-4bit",
        max_seq_length=2048,
        load_in_4bit=True,
        output_dir="./training_output",
        hub_model_id=None
    ):
        self.base_model_name = base_model_name
        self.max_seq_length = max_seq_length
        self.load_in_4bit = load_in_4bit
        self.output_dir = output_dir
        self.hub_model_id = hub_model_id or "your-username/gemma-2b-custom"
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Login to Hugging Face
        notebook_login()

    def load_and_check_data(self, train_path, valid_path):
        """Load and validate the dataset"""
        logger.info("Checking and loading dataset...")
        
        for path in [train_path, valid_path]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"File not found: {path}")
        
        dataset = load_dataset("json", 
                             data_files={
                                 "train": train_path, 
                                 "validation": valid_path
                             })
        
        logger.info(f"Dataset loaded successfully. Train size: {len(dataset['train'])}, "
                   f"Validation size: {len(dataset['validation'])}")
        return dataset

    def setup_model(self):
        """Set up the model and tokenizer"""
        logger.info("Setting up model and tokenizer...")
        
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=self.base_model_name,
            max_seq_length=self.max_seq_length,
            dtype=None,
            load_in_4bit=self.load_in_4bit
        )

        # PEFT configuration
        model = FastLanguageModel.get_peft_model(
            model,
            r=16,
            lora_alpha=32,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
            lora_dropout=0.05,
            bias="none",
            use_gradient_checkpointing=True,
            random_state=42,
            max_seq_length=self.max_seq_length
        )

        return model, tokenizer

    def get_training_args(self):
        """Set up training arguments"""
        return TrainingArguments(
            output_dir=self.output_dir,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=16,
            optim="adamw_torch",
            save_steps=500,
            logging_steps=100,
            learning_rate=2e-4,
            fp16=True,
            gradient_checkpointing=True,
            max_steps=100,
            warmup_ratio=0.03,
            group_by_length=True,
            lr_scheduler_type="constant",
            push_to_hub=True,
            hub_model_id=self.hub_model_id,
        )

    def train(self, dataset):
        """Execute the training process"""
        logger.info("Starting training process...")
        
        model, tokenizer = self.setup_model()
        training_args = self.get_training_args()

        def formatting_func(example):
            return example['text']

        trainer = SFTTrainer(
            model=model,
            train_dataset=dataset["train"],
            eval_dataset=dataset["validation"],
            tokenizer=tokenizer,
            args=training_args,
            packing=False,
            max_seq_length=256,
            formatting_func=formatting_func
        )

        logger.info("Training started...")
        trainer.train()
        
        # Save the model
        logger.info("Saving model...")
        save_path = os.path.join(self.output_dir, "final_model")
        unsloth_save_model(model, tokenizer, save_path)
        
        return save_path

    def convert_to_gguf(self, model_path):
        """Convert the trained model to GGUF format"""
        logger.info("Starting GGUF conversion...")
        
        gguf_output_dir = os.path.join(self.output_dir, "gguf")
        os.makedirs(gguf_output_dir, exist_ok=True)
        gguf_model_path = os.path.join(gguf_output_dir, "model.gguf")

        # Install llama.cpp if needed
        subprocess.run(["pip", "install", "llama-cpp-python"])

        # Convert to GGUF
        conversion_command = [
            "python3", "-m", "llama_cpp.convert_llama_weights_to_gguf",
            "--outfile", gguf_model_path,
            "--input-dir", model_path,
            "--model-name", os.path.basename(self.hub_model_id)
        ]
        
        subprocess.run(conversion_command)
        
        # Create and upload to HuggingFace
        gguf_repo_name = f"{self.hub_model_id}-gguf"
        try:
            create_repo(gguf_repo_name, private=False)
        except Exception as e:
            logger.warning(f"Repository creation warning: {e}")

        logger.info(f"Uploading GGUF model to {gguf_repo_name}...")
        upload_file(
            path_or_fileobj=gguf_model_path,
            path_in_repo="model.gguf",
            repo_id=gguf_repo_name,
            repo_type="model"
        )

        return gguf_model_path, gguf_repo_name

    def test_model(self, gguf_model_path):
        """Test the converted GGUF model"""
        logger.info("Testing GGUF model...")
        
        model = CTModel.from_pretrained(gguf_model_path)
        test_prompt = "Hello, how are you?"
        response = model(test_prompt, max_new_tokens=50)
        
        logger.info(f"Test response: {response}")
        return response

def main():
    # Configuration
    trainer = ModelTrainer(
        base_model_name="unsloth/gemma-2b-bnb-4bit",
        hub_model_id="gunzzz24/gemma-2b-custom"  # Replace with your username
    )
    
    # Training
    try:
        # Load dataset
        dataset = trainer.load_and_check_data(
            train_path="train_data/train.jsonl",
            valid_path="train_data/valid.jsonl"
        )
        
        # Train model
        model_path = trainer.train(dataset)
        
        # Convert to GGUF
        gguf_model_path, gguf_repo_name = trainer.convert_to_gguf(model_path)
        
        # Test the model
        test_response = trainer.test_model(gguf_model_path)
        
        logger.info(f"""
        Training completed successfully!
        - Fine-tuned model saved to: {model_path}
        - GGUF model saved to: {gguf_model_path}
        - GGUF model uploaded to: https://huggingface.co/{gguf_repo_name}
        """)
        
    except Exception as e:
        logger.error(f"Error during training: {str(e)}")
        raise

if __name__ == "__main__":
    main()

INFO:__main__:Checking and loading dataset...
INFO:__main__:Dataset loaded successfully. Train size: 40, Validation size: 10
INFO:__main__:Starting training process...
INFO:__main__:Setting up model and tokenizer...


==((====))==  Unsloth 2024.10.3: Fast Gemma patching. Transformers = 4.45.2.
   \\   /|    GPU: NVIDIA GeForce GTX 1650. Max memory: 4.0 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers via:
`pip uninstall transformers -y && pip install --upgrade --no-cache-dir "git+https://github.com/huggingface/transformers.git"`
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.10.3 patched 18 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
INFO:__main__:Training started...
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 40 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 16
\        /    Total batch size = 16 | Total steps = 100
 "-____-"     Number of trainable parameters = 3,686,400


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers and Unsloth!


Step,Training Loss


In [ ]:
from ctransformers import AutoModelForCausalLM

# Load from HuggingFace
model = AutoModelForCausalLM.from_pretrained("gunzzz24/gemma-2b-custom-gguf")

# Generate text
response = model("Hi, today is a sunny day!", max_new_tokens=100)
print(response)

### reformatting to correct dataset

In [1]:
!pip install spacy PyPDF2
!python -m spacy download en_core_web_sm

  Using cached pypdf2-3.0.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached typer-0.12.5-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached wrapt-1.16.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 31.7 MB/s eta 0:00:00a 0:00:01
Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 

In [3]:
import os
import re
import json
import PyPDF2
import spacy
from spacy.matcher import Matcher

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def extract_text_from_pdf(pdf_path):
    """Extract raw text from PDF"""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

def extract_structured_info(text):
    """Extract structured information from resume text"""
    doc = nlp(text)
    
    # Define patterns for different sections
    patterns = {
        "education": [
            [{"LOWER": "education"}],
            [{"TEXT": {"REGEX": "^(MSc|BSc|PhD|Master|Bachelor)"}}]
        ],
        "experience": [
            [{"LOWER": "experience"}],
            [{"LOWER": "work"}, {"LOWER": "experience"}]
        ],
        "skills": [
            [{"LOWER": "skills"}],
            [{"LOWER": "technical"}, {"LOWER": "skills"}]
        ],
        "achievements": [
            [{"LOWER": "achievements"}],
            [{"LOWER": "key"}, {"LOWER": "achievements"}]
        ]
    }
    
    # Initialize matcher
    matcher = Matcher(nlp.vocab)
    for key, patterns_list in patterns.items():
        matcher.add(key, patterns_list)
    
    # Find all matches
    matches = matcher(doc)
    sections = {}
    current_section = None
    
    # Sort matches by their appearance in text
    sorted_matches = sorted(matches, key=lambda x: x[1])
    
    # Extract text between matches
    for i, (match_id, start, end) in enumerate(sorted_matches):
        section_name = doc.vocab.strings[match_id]
        if i < len(sorted_matches) - 1:
            next_start = sorted_matches[i + 1][1]
            section_text = doc[start:next_start].text.strip()
        else:
            section_text = doc[start:].text.strip()
        sections[section_name] = section_text
    
    return sections

def extract_dates_and_orgs(text):
    """Extract dates and organizations from text"""
    doc = nlp(text)
    dates = []
    orgs = []
    
    for ent in doc.ents:
        if ent.label_ == "DATE":
            dates.append(ent.text)
        elif ent.label_ == "ORG":
            orgs.append(ent.text)
    
    return dates, orgs

def format_resume_context(sections):
    """Format extracted sections into a structured context"""
    context = "Resume Information:\n\n"
    
    if "education" in sections:
        context += "Education:\n"
        dates, orgs = extract_dates_and_orgs(sections["education"])
        for org in orgs:
            relevant_dates = [d for d in dates if sections["education"].find(d) < sections["education"].find(org) + len(org)]
            if relevant_dates:
                context += f"- {org}: {', '.join(relevant_dates)}\n"
    
    if "experience" in sections:
        context += "\nExperience:\n"
        dates, orgs = extract_dates_and_orgs(sections["experience"])
        for org in orgs:
            relevant_dates = [d for d in dates if sections["experience"].find(d) < sections["experience"].find(org) + len(org)]
            if relevant_dates:
                context += f"- {org}: {', '.join(relevant_dates)}\n"
    
    if "skills" in sections:
        context += "\nSkills:\n"
        # Split skills into a list and format
        skills_text = sections["skills"].replace(",", "\n-")
        context += f"- {skills_text}\n"
    
    if "achievements" in sections:
        context += "\nKey Achievements:\n"
        # Split achievements into bullet points
        achievements = sections["achievements"].split(".")
        for achievement in achievements:
            if achievement.strip():
                context += f"- {achievement.strip()}\n"
    
    return context

def reformat_training_data(original_jsonl, output_jsonl, resume_context):
    """Reformat existing training data to include resume context"""
    reformatted_data = []
    
    with open(original_jsonl, 'r', encoding='utf-8') as file:
        for line in file:
            entry = json.loads(line)
            text = entry['text']
            
            # Remove tags and split into instruction and response
            text = text.replace('<s>', '').replace('</s>', '')
            parts = text.split('[/INST]')
            
            if len(parts) == 2:
                input_text = parts[0].replace('[INST]', '').strip()
                output_text = parts[1].strip()
                
                # Create new formatted entry
                formatted_entry = {
                    "context": resume_context,
                    "instruction": "Based on the provided resume information, answer the following question:",
                    "input": input_text,
                    "output": output_text
                }
                reformatted_data.append(formatted_entry)
    
    # Save reformatted data
    with open(output_jsonl, 'w', encoding='utf-8') as file:
        for entry in reformatted_data:
            json.dump(entry, file)
            file.write('\n')
    
    return reformatted_data

def main():
    # File paths
    pdf_path = "raw_data/UK_Resume_Alexis_BALAYRE.pdf"
    train_file = "train_data/train.jsonl"
    valid_file = "train_data/valid.jsonl"
    reformatted_train = "reformatted_train.jsonl"
    reformatted_valid = "reformatted_valid.jsonl"
    
    # Extract and process resume
    print("Extracting text from PDF...")
    resume_text = extract_text_from_pdf(pdf_path)
    
    print("Extracting structured information...")
    sections = extract_structured_info(resume_text)
    
    print("Formatting resume context...")
    resume_context = format_resume_context(sections)
    
    # Save formatted context for reference
    with open("resume_context.txt", 'w', encoding='utf-8') as f:
        f.write(resume_context)
    
    print("Reformatting training data...")
    # Reformat training and validation data
    reformat_training_data(train_file, reformatted_train, resume_context)
    reformat_training_data(valid_file, reformatted_valid, resume_context)
    
    print(f"Formatted data saved to {reformatted_train} and {reformatted_valid}")
    print(f"Resume context saved to resume_context.txt")

if __name__ == "__main__":
    main()

Extracting text from PDF...
Extracting structured information...
Formatting resume context...
Reformatting training data...
Formatted data saved to reformatted_train.jsonl and reformatted_valid.jsonl
Resume context saved to resume_context.txt
